In [2]:
# Importación de librerías necesarias para el proyecto

# Conexión y manejo de bases de datos PostgreSQL
# -----------------------------------------------------------------------
import psycopg2                                 # Conexión con bases de datos PostgreSQL
from psycopg2 import OperationalError, errorcodes, errors  # Manejo de errores específicos de psycopg2

# Manipulación de fechas y tiempos
# -----------------------------------------------------------------------
import datetime                                 # Manipulación de fechas y tiempos

# Manipulación de DataFrames y variables de entorno
# -----------------------------------------------------------------------
import pandas as pd                             # Manipulación y análisis de datos en estructuras DataFrame
import dotenv                                   # Manejo de variables de entorno (.env)
import os                                       # Interacción con el sistema operativo (lectura de variables de entorno)
from datetime import datetime                   # Importación específica de datetime para facilitar manejo de fechas

# Importación de módulos desde rutas recursivas
# -----------------------------------------------------------------------
import sys
sys.path.append("../")                          # Añadir rutas externas para importar módulos desde carpetas superiores
from src.SupportDB import (
    conectarse_a_bd,
    modificar_bd,
    insertar_muchos_datos_bd,
    generar_tupla
)                                              # Funciones de soporte para manipulación y modificación en la base de datos


# Ver datos


In [3]:
detalles_artista = pd.read_csv("../datos/detalles_artistas.csv")
detalles_peliculas = pd.read_csv("../datos/detalles_peliculas.csv")
pelicula_artista = pd.read_csv("../datos/pelicula_artista.csv")
peliculas = pd.read_csv("../datos/peliculas.csv")


In [4]:
detalles_artista.sample()

,actr,año_nac,conocido_por,que_hace,premios
5357,Kristy O'Leary,9999,"'Romance', 'The Last Class', 'Under the Volcan...","Actress, Make-Up Department, Producer",no tiene premios


In [5]:
detalles_peliculas.sample()

,titulo,cali_imdb,direc_imbd,guion_imbd,argum_imbd,durac_imbd,id_pelicula
102,Gigolo,5.5,Desconocido,Desconocido,Desconocido,1h 38min,tt0101952


In [6]:
pelicula_artista.sample()

,Movie ID,Actriz_Actor
9143,tt0109800,Karin Viard


In [7]:
peliculas.sample()

,Tipo,Titulo,Año,Mes,Id,generos
886,Movie,Rasigan,1994,7,tt0331642,Romance


### Unimos detalles artista con pelicula artista

In [8]:
pelicula_artista = pelicula_artista.rename(columns={"Actriz_Actor":"actr"})
pelicula_artista["actr"] = pelicula_artista["actr"].str.lower()
detalles_artista["actr"] = detalles_artista["actr"].str.lower()
peli_artista = pd.merge(pelicula_artista,detalles_artista,on="actr",how="left")

In [9]:
peli_artista.dropna(inplace=True)

In [10]:
peli_artista.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14527 entries, 0 to 14528
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Movie ID      14527 non-null  object 
 1   actr          14527 non-null  object 
 2   año_nac       14527 non-null  float64
 3   conocido_por  14527 non-null  object 
 4   que_hace      14527 non-null  object 
 5   premios       14527 non-null  object 
dtypes: float64(1), object(5)
memory usage: 794.4+ KB


In [11]:
peli_artista["año_nac"] = peli_artista["año_nac"].astype(int)
peli_artista.sample()

,Movie ID,actr,año_nac,conocido_por,que_hace,premios
933,tt2698106,alessandro de marco,1979,"'Kingsman: El círculo de oro', 'Espías', 'Rush...",Actor,no tiene premios


In [12]:
peli_artista.duplicated().sum()

np.int64(3298)

### Unimos los detalles pelicula con peliculas

In [13]:
peliculas.sample()

,Tipo,Titulo,Año,Mes,Id,generos
787,Short,I Want Mommy,2019,7,tt10773386,Horror


In [14]:
detalles_peliculas.sample()

,titulo,cali_imdb,direc_imbd,guion_imbd,argum_imbd,durac_imbd,id_pelicula
488,Skin Deep,NaN,Desconocido,Desconocido,Desconocido,9m,tt2008636


In [15]:
peliculas["Id"] = peliculas["Id"].str.lower()
detalles_peliculas["id_pelicula"] = detalles_peliculas["id_pelicula"].str.lower()
detalles_peliculas = detalles_peliculas.rename(columns={"id_pelicula":"Id"})
pelis = pd.merge(peliculas,detalles_peliculas,on="Id",how="left")
pelis.to_pickle("../datos/02_tabla_para_sist_recomendacion.pkl")
pelis.sample()

,Tipo,Titulo,Año,Mes,Id,generos,titulo,cali_imdb,direc_imbd,guion_imbd,argum_imbd,durac_imbd
1073,Movie,¡Atraco!,2012,8,tt2203951,Thriller,Â¡Atraco!,6.3,Desconocido,Desconocido,Desconocido,1h 51min


In [16]:
peli_artista.sample()

,Movie ID,actr,año_nac,conocido_por,que_hace,premios
9186,tt0213733,eszter ónodi,1973,"'Boldog születésnapot!', 'Aglaja', 'Az alkimis...","Actress, Producer, Writer",no tiene premios


In [17]:
key = os.getenv("sql")
user = "postgres"
db = "prueba_tecnica"

In [18]:
conexion = conectarse_a_bd(db,user,key)
cursor = conexion.cursor()

Conectado a la base de datos: prueba_tecnica


# T

In [19]:
# conexion = conectarse_a_bd(db,user,key)
# query = '''CREATE TABLE IF NOT EXISTS supermercados (
#             id_supermercado SERIAL PRIMARY KEY,
#             supermercado VARCHAR(100) NOT NULL);'''
# modificar_bd(conexion,query)